In [13]:
import json

fp = open("folding_config/KV260_build_base.json")
f = json.load(fp)
max_usage = {"BRAM_18K": 144, "LUT" : 117120, "URAM": 64, "DSP": 1248}
total = {}

ram_style = ["distributed","auto", "block", 'ultra']
resType = [""]

for r in ram_style:
    fn = {}
    
    for k,v in f.items():
        if "MatrixVectorActivation" in k:        
            if int(k.split("_")[1]) > 6:
                v["ram_style"] = r
                v["resType"] = 'dsp'
        if "SIMD" in v:
            v["SIMD"] = 1
        if "PE" in v:
            v["PE"] = 1
        fn[k] = v
        
    fold_config = "KV260_build_%s.json" %r
    fnp = open("folding_config/%s"%fold_config,'w')
    json.dump(fn,fnp)
    fnp.close()

    ! python "build_resnet-18.py" --ram_style={r} --fold_file={fold_config}
    report = "build-KV260_SOM/resnet18_img_3-%s/report/estimate_layer_resources.json" %r
    
    total[r] = json.load(open(report))["total"]
    print(total)
    
fp.close()


Previous run results deleted!
folding_config/KV260_build_distributed.json
Building dataflow accelerator from models/resnet18_img_3.onnx
/workspace/qonnx/src/qonnx/core/modelwrapper.py:93: UserWarning: Some old-style domain attributes were automatically converted to new-style,
                i.e. domain=finn to domain=qonnx.custom_op.<general|fpgadataflow|...>
  warnings.warn(
Intermediate outputs will be generated in /workspace/results/KV260_SOM/resnet18_img_3-distributed
Final outputs will be generated in build-KV260_SOM/resnet18_img_3-distributed
Build log is at build-KV260_SOM/resnet18_img_3-distributed/build_dataflow.log
Running step: step_resnet18_tidy [1/6]
Running step: step_resnet18_streamline [2/6]
Running step: step_resnet18_convert_to_hls [3/6]
Running step: step_create_dataflow_partition [4/6]
Running step: step_apply_folding_config [5/6]
Running step: step_generate_estimate_reports [6/6]
Completed successfully
{'distributed': {'BRAM_18K': 24.0, 'LUT': 528449.0, 'URAM': 0.

In [14]:
import pandas as pd

df = pd.DataFrame(total)
df
df.to_csv("ram_style_dsp.csv")